In [ ]:
import os
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import riskfolio as rp
from binance.client import Client
from dotenv import load_dotenv

# 1) Load env and init client
load_dotenv()
api_key    = os.getenv("BINANCE_API_KEY")
api_secret = os.getenv("BINANCE_API_SECRET")
client = Client(api_key, api_secret)

# 2) Download historical closes for top 20 USDT pairs
assets = [
    "BTCUSDT","ETHUSDT","XRPUSDT","BNBUSDT","SOLUSDT",
    "USDCUSDT","ADAUSDT","DOGEUSDT","DOTUSDT","LTCUSDT",
    "BCHUSDT","LINKUSDT","XMRUSDT","XLMUSDT","XTZUSDT",
    "TRXUSDT","NEOUSDT","AVAXUSDT","MATICUSDT","FTMUSDT"
]

end_date   = datetime.today()
start_date = end_date - timedelta(days=730)

data = pd.DataFrame()
for asset in assets:
    try:
        print(f"Downloading data for {asset}")
        candles = client.get_historical_klines(
            asset,
            Client.KLINE_INTERVAL_1DAY,
            start_str=str(start_date),
            end_str=str(end_date)
        )
        df = pd.DataFrame(candles, columns=[
            "timestamp","open","high","low","close","volume",
            "close_time","quote_asset_volume","number_of_trades",
            "taker_buy_base_asset_volume","taker_buy_quote_asset_volume","ignore"
        ])
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
        df = df.set_index("timestamp")[["close"]].astype(float)
        df.rename(columns={"close": asset}, inplace=True)
        data = pd.concat([data, df], axis=1)
        print(f"  → {asset} done")
    except Exception as e:
        print(f"Failed for {asset}: {e}")

# 3) Clean & compute returns
returns = data.dropna().pct_change().dropna()

# 4) Plot clusters
ax = rp.plot_clusters(
    returns      = returns,
    codependence = "pearson",
    linkage      = "ward",
    k            = None,
    max_k        = 10,
    leaf_order   = True,
    dendrogram   = True
)

fig = ax.get_figure()
fig.set_size_inches(12, 8)           # give the plot more canvas
fig.set_constrained_layout(True)     # <-- replaces tight_layout()

# make tick labels readable
plt.setp(ax.get_xticklabels(), rotation=45, ha='right', fontsize=9)

# title with extra padding
ax.set_title("Hierarchical clustering of top-20 USDT pairs",
             fontsize=12, pad=18)

fig.savefig("clustered_correlation.png",
            dpi=300, bbox_inches="tight")  # bbox_inches still fine

plt.show()
plt.close(fig)

  → BTCUSDT done
  → ETHUSDT done
  → XRPUSDT done
  → BNBUSDT done
  → SOLUSDT done
  → USDCUSDT done
  → ADAUSDT done
  → DOGEUSDT done
  → DOTUSDT done
  → LTCUSDT done
  → BCHUSDT done
  → LINKUSDT done
  → XMRUSDT done
  → XLMUSDT done
  → XTZUSDT done
  → TRXUSDT done
  → NEOUSDT done
  → AVAXUSDT done
  → MATICUSDT done
  → FTMUSDT done


RuntimeError: Colorbar layout of new layout engine not compatible with old engine, and a colorbar has been created.  Engine not changed.